In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:

# 1. 全局配置
# --------------------------
SECOND_HAND_BASE_URL = "https://zhangjiakou.esf.fang.com/house-a014963/"  # 张家口下花园二手房基础URL
TARGET_PAGES = 10  # 目标爬取总页数
WAIT_TIME = 15  # 单次页面加载等待时间（秒，网络慢可调大）
RETRY_DELAY = 2 # 页面爬取失败后的重试间隔（秒）

# --------------------------
# 2. 初始化Chrome浏览器
# --------------------------
def init_browser():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()  # 避免元素因窗口过小被隐藏
    return driver

In [3]:

# 3. 提取单条二手房信息
# --------------------------
def extract_second_hand_info(house_dl):
    """
    提取单条二手房核心信息：房屋面积、总价、单价、户型
    规则：
    - 面积：p标签，class=tel_shop，第二个字段
    - 总价/单价：dd标签，class=price_right（总价红体span，单价无class含"元/㎡"）
    - 户型：p标签，class=tel_shop，第一个字段
    """
    house_info = {"房源类型": "二手房"}
    
    # 提取房屋面积（p.tel_shop，第二个字段）
    area_line_elem = house_dl.find("p", class_="tel_shop")
    if area_line_elem:
        area_line_text = area_line_elem.get_text(strip=True)
        area_line_fields = area_line_text.split("|")
        # 提取户型（第一个字段）
        house_info["户型"] = area_line_fields[0] if len(area_line_fields) >= 1 else "无"
        print("------ 户型：", house_info["户型"])
        # 提取房屋面积（第二个字段）
        house_info["房屋面积"] = area_line_fields[1] if len(area_line_fields) >= 2 else "无"
        print("------ 房屋面积：", house_info["房屋面积"])
    else:
        house_info["户型"] = "无"
        house_info["房屋面积"] = "无"
    # 提取总价和单价（dd.price_right逻辑）
    price_right = house_dl.find("dd", class_="price_right")
    if price_right:
        # 总价：红色粗体span
        total_price = price_right.find("span", class_="red")
        house_info["总价"] = total_price.get_text(strip=True) if total_price else "无"
        print("price:", house_info["总价"])
        # 单价：无特定class，含"元/㎡"的span
        unit_price = price_right.find("span", class_=None)
        house_info["单价"] = unit_price.get_text(strip=True) if (unit_price and "元/㎡" in unit_price.get_text()) else "无"
        print("unit price:", house_info["单价"])
    else:
        house_info["总价"] = "无"
        house_info["单价"] = "无"
    # 返回存储信息的字典
    return house_info


In [4]:
# --------------------------
# 4. 单页爬取
# --------------------------
def crawl_single_page(driver, current_url):
    """单页爬取：最多重试3次，仍无数据则返回空列表"""
    max_retries = 3  # 最大重试次数
    retry_count = 0   # 已重试次数

    while retry_count < max_retries:
        retry_count += 1
        try:
            driver.get(current_url)
            print(f"[第{retry_count}次重试] 加载页面：{current_url}")
            
            # 等待核心元素加载
            WebDriverWait(driver, WAIT_TIME).until(
                EC.presence_of_element_located((By.XPATH, "//dl[@dataflag='bg']"))
            )
            time.sleep(2)  # 等待动态内容
            
            # 解析页面并提取数据
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "lxml")
            house_dls = soup.find_all("dl", attrs={"dataflag": "bg"})
            
            current_page_data = []
            for house_dl in house_dls:
                house_info = extract_second_hand_info(house_dl)
                # 过滤有效数据（面积和总价都不为空）
                if house_info["房屋面积"] != "无" and house_info["总价"] != "无":
                    current_page_data.append(house_info)
            
            if current_page_data:
                print(f"✅ 第{retry_count}次重试成功，获取{len(current_page_data)}条数据")
                return current_page_data
            else:
                print(f"⚠️ 第{retry_count}次重试无有效数据，继续...")
                time.sleep(RETRY_DELAY)  # 重试间隔

        except TimeoutException:
            print(f"⚠️ 第{retry_count}次重试：页面加载超时，继续...")
            time.sleep(RETRY_DELAY)
        except Exception as e:
            print(f"⚠️ 第{retry_count}次重试：错误 {str(e)[:60]}，继续...")
            time.sleep(RETRY_DELAY)
    
    # 超过最大重试次数，返回空列表
    print(f"❌ 重试超过{max_retries}次，无有效房源")
    return []

In [5]:
# --------------------------
# 5. 多页爬取主逻辑
# --------------------------
def crawl_multi_page(driver):
    all_house_data = []
    print(f"开始爬取，最多{TARGET_PAGES}页（房源为空则停止）")

    for page in range(1, TARGET_PAGES + 1):
        print(f"\n📄 处理第{page}/{TARGET_PAGES}页")
        # 生成当前页URL
        current_url = SECOND_HAND_BASE_URL if page == 1 else f"{SECOND_HAND_BASE_URL}i{30 + page}/"
        
        # 爬取当前页（最多重试3次）
        current_page_data = crawl_single_page(driver, current_url)
        
        if not current_page_data:
            print(f"⚠️ 第{page}页无有效房源，停止翻页")
            break  # 停止后续页面爬取
        
        # 添加数据到总列表
        for data in current_page_data:
            data["来源页"] = f"第{page}页"
        all_house_data.extend(current_page_data)

    df = pd.DataFrame(all_house_data)
    print(f"爬取完成，共{len(df)}条数据")
    return df


# --------------------------
# 6. 保存数据到Excel
# --------------------------
def save_to_excel(second_hand_df):
    
    user_home = os.path.expanduser("~")
    excel_path = os.path.join(user_home, "Downloads", "zhangjiakou_esf_xiahuayuan_data.xlsx")
    
    try:
        # 使用openpyxl引擎保存Excel
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
            second_hand_df.to_excel(writer, sheet_name="张家口下花园二手房", index=False)
        print(f"\n📁 数据保存成功！Excel文件路径：")
        print(f"   {excel_path}")
    except Exception as e:
        print(f"\n❌ 保存Excel失败：{str(e)}")


# --------------------------
# 7. 主执行流程
# --------------------------
if __name__ == "__main__":
    # 初始化浏览器
    driver = init_browser()
    try:
        # 执行多页爬取
        second_hand_data = crawl_multi_page(driver)
        # 保存爬取到的数据
        if not second_hand_data.empty:
            save_to_excel(second_hand_data)
        else:
            print("\n⚠️ 未获取到任何二手房数据（可能URL或定位规则错误）")
    finally:
        # 无论爬取是否成功，都关闭浏览器释放资源
        driver.quit()
        print(f"\n{'='*60}")
        print("Chrome浏览器已关闭，爬取流程结束")
        print(f"{'='*60}")

开始爬取，最多10页（房源为空则停止）

📄 处理第1/10页
[第1次重试] 加载页面：https://zhangjiakou.esf.fang.com/house-a014963/
------ 户型： 3室2厅
------ 房屋面积： 103㎡
price: 50万
unit price: 4854元/㎡
------ 户型： 3室1厅
------ 房屋面积： 98㎡
price: 50万
unit price: 5102元/㎡
------ 户型： 3室2厅
------ 房屋面积： 111.1㎡
price: 95万
unit price: 8551元/㎡
------ 户型： 3室2厅
------ 房屋面积： 99㎡
price: 47万
unit price: 4747元/㎡
------ 户型： 3室2厅
------ 房屋面积： 160㎡
price: 55.8万
unit price: 3488元/㎡
------ 户型： 3室2厅
------ 房屋面积： 95.18㎡
price: 44.8万
unit price: 4707元/㎡
------ 户型： 3室1厅
------ 房屋面积： 99.66㎡
price: 65万
unit price: 6523元/㎡
------ 户型： 2室1厅
------ 房屋面积： 88.22㎡
price: 46万
unit price: 5215元/㎡
------ 户型： 2室1厅
------ 房屋面积： 32.21㎡
price: 16.8万
unit price: 5216元/㎡
------ 户型： 3室2厅
------ 房屋面积： 133.55㎡
price: 66万
unit price: 4942元/㎡
------ 户型： 2室1厅
------ 房屋面积： 31.58㎡
price: 13万
unit price: 4117元/㎡
------ 户型： 2室1厅
------ 房屋面积： 83.48㎡
price: 30万
unit price: 3593元/㎡
------ 户型： 3室2厅
------ 房屋面积： 96.83㎡
price: 44万
unit price: 4545元/㎡
------ 户型： 2室1厅
------ 房屋面积： 42.4㎡
pric